In [1]:
from qdrant_client import QdrantClient
import openai

We need embedding function to translate the user input query to points for vector DB storage

In [2]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )
    return response.data[0].embedding

In [4]:
qdrant_client = QdrantClient(url="http://localhost:6333")


We first define the retrieval function

In [3]:

def retrieve_data(query, qdrant_client, k=5):
    query_embedding = get_embedding(query)
    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []
    retrieved_context_ratings = []
    retrieved_context_prices =[]
    retrieved_context_images = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        retrieved_context_ratings.append(result.payload["average_rating"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "retrieved_context_ratings": retrieved_context_ratings,
        "similarity_scores": similarity_scores,
    }

In [5]:
retrieved_context = retrieve_data("What kind of earphones can I get?", qdrant_client, k=10)

In [6]:
retrieved_context

{'retrieved_context_ids': ['B0CBMPG524',
  'B0B9FTVL58',
  'B0C142QS8X',
  'B0C6K1GQCF',
  'B0BNHVLF7G',
  'B0B14HTZ59',
  'B09WCFC5D9',
  'B0B67ZFRPC',
  'B0B7495RL6',
  'B0BS15TRJ3'],
 'retrieved_context': ['Open Ear Headphones, Bluetooth 5.3 Earbuds with 60H Playtime IPX7 Waterproof Wireless Earbuds Immersive Premium Sound True Wireless Open Ear Earbuds with Earhooks for Running, Walking and Workouts 【Open-ear Design Headphones】Feature with a new generation of true open-ear wireless earbuds design, the headphones can rest gently and firmly fit your ears without entering your ear canal, which will reduce stress and hearing loss after extended wear. There is no pinching of the auricle, no blockage of the ear canal, and no pain or damage to hearing. 【Powerful Stereo Sound】Equipped with 16.2 millimeters vibrating diaphragm speaker driver, bluetooth headphones providing pure balanced audio and clarity output for all music genres with soft audio and immerse yourself in the wonderful world

Format the output for LLM (single string)

In [7]:
def process_context(context):

    formatted_context = ""

    for id, chunk, rating in zip(context["retrieved_context_ids"], context["retrieved_context"], context["retrieved_context_ratings"]):
        formatted_context += f"- ID: {id}, rating: {rating}, description: {chunk}\n"

    return formatted_context

In [8]:
preprocessed_context = process_context(retrieved_context)

In [10]:
print(preprocessed_context)

- ID: B0CBMPG524, rating: 4.4, description: Open Ear Headphones, Bluetooth 5.3 Earbuds with 60H Playtime IPX7 Waterproof Wireless Earbuds Immersive Premium Sound True Wireless Open Ear Earbuds with Earhooks for Running, Walking and Workouts 【Open-ear Design Headphones】Feature with a new generation of true open-ear wireless earbuds design, the headphones can rest gently and firmly fit your ears without entering your ear canal, which will reduce stress and hearing loss after extended wear. There is no pinching of the auricle, no blockage of the ear canal, and no pain or damage to hearing. 【Powerful Stereo Sound】Equipped with 16.2 millimeters vibrating diaphragm speaker driver, bluetooth headphones providing pure balanced audio and clarity output for all music genres with soft audio and immerse yourself in the wonderful world of music. 【Hear Your Surroundings】Open earbuds that rest on your ears without covering them, you can hear your music and your surroundings at the same time. Whether 

In [13]:


def build_prompt(preprocessed_context, question):

    prompt = f"""
You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
{preprocessed_context}

Question:
{question}
"""

    return prompt



In [14]:

prompt = build_prompt(preprocessed_context, "What kind of earphones can I get?")

In [15]:
print(prompt)


You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
- ID: B0CBMPG524, rating: 4.4, description: Open Ear Headphones, Bluetooth 5.3 Earbuds with 60H Playtime IPX7 Waterproof Wireless Earbuds Immersive Premium Sound True Wireless Open Ear Earbuds with Earhooks for Running, Walking and Workouts 【Open-ear Design Headphones】Feature with a new generation of true open-ear wireless earbuds design, the headphones can rest gently and firmly fit your ears without entering your ear canal, which will reduce stress and hearing loss after extended wear. There is no pinching of the auricle, no blockage of the ear canal, and no pain or damage to hearing. 【Powerful Stereo Sound】Equipped with 16.2 millimeters vibrating diaphragm speaker driver, bluetooth headp

Generate answer for the prompt using LLM

In [16]:
def generate_answer(prompt):

    response = openai.chat.completions.create(
        model="gpt-5-nano",
        messages=[{"role": "system", "content": prompt}],
        reasoning_effort="minimal"
    )

    return response.choices[0].message.content


In [17]:
print(generate_answer(prompt))

You can get a variety of wireless/open-ear and in-ear earphones from the available products, including:

- Open-ear wireless earbuds with earhooks (comfortable, open-ear design, IPX7, long playback up to 60 hours with case)
- Wireless in-ear earbuds with Bluetooth 5.3, IPX7, and features like LED power display and long battery life
- Bone conduction open-ear headphones for sport use
- Kids' over-ear/headphones with volume limiting
- Gaming/low-latency true wireless earbuds with game/music modes (Bluetooth 5.2)
- Various accessories like ear tip replacements and cleaning kits

If you tell me your priorities (price, open-ear vs. in-ear, battery life, waterproofing, or kids’ use), I can narrow down the best options from the available products.


Combined RAG pipeline

In [18]:
def rag_pipeline(question, top_k=5):

    ## Connect to Vector DB
    qdrant_client = QdrantClient(url="http://localhost:6333")

    ## Retrieve data from Qdrant DB in proper format
    retrieved_context = retrieve_data(question, qdrant_client, top_k)

    ## For LLM, format the retrieved context to a single string with clear demarcation using ":""
    preprocessed_context = process_context(retrieved_context)

    ## Build prompt for LLM with the preprocessed context inline
    prompt = build_prompt(preprocessed_context, question)

    ## Use LLM to generate answer relevant to the context
    answer = generate_answer(prompt)

    return answer

In [19]:
print(rag_pipeline("What kind of earphones can I get with ratings above 4.5?"))


From the available products, the earphones with ratings above 4.5 are:

- B0CBMPG524: Open Ear Headphones with a rating of 4.4 (not above 4.5) → does not qualify.
- B0C142QS8X: TUNEAKE Kids Headphones with rating 4.5 exactly (not above 4.5) → does not qualify.
- B0BNHVLF7G: Siniffo Upgraded Bone Conduction Headphones with rating 4.0 → does not qualify.
- B0C6K1GQCF: TELSOR Wireless Earbuds with rating 4.3 → does not qualify.
- B0B9FTVL58: Wireless Earbuds with rating 4.2 → does not qualify.

None of the listed items have a rating above 4.5. If you’re looking for earphones with ratings higher than 4.5, they aren’t available in the current available products.
